In [4]:
import random
from jinja2 import Environment, Template, meta
import sys
from itertools import product
from datasets import load_dataset
import polars as pl
from vllm import LLM, SamplingParams
from typing import Dict
import numpy as np
import ray
from transformers import AutoTokenizer
from typing import Any, Dict

In [5]:
# model_name = 'mistralai/Mistral-7B-Instruct-v0.2'
model_name = '/home/lawrence/.cache/huggingface/hub/models--mistralai--Mistral-7B-Instruct-v0.2/snapshots/41b61a33a2483885c981aa79e0df6b32407ed873'
# model_name = 'google/gemma-7b-it'

tokenizer = AutoTokenizer.from_pretrained(
    model_name
) 

## import the dataset

## load the dataset into ray

In [6]:
def get_chat_input(row: Dict[str, Any]) -> Dict[str, Any]:
    
    #original text prefix
    orig_prefix = "Original Text:"
    #modified text prefix
    rewrite_prefix = "Rewritten Text:"
    # response start
    response_start = "The prompt was:"

    sys_prompt = """You are an expert in "Reverse Prompt Engineering". You are able to reverse-engineer prompts used to rewrite text.\n\nI will be providing you with an "original text" and "rewritten text". Please try to be as specific as possible and come up with a prompt that is based on the tone, style, and any other properties you consider relevant."""

    #first example
    ex_text_1 = "The aroma of freshly baked bread wafted through the bustling marketplace, mingling with the shouts of vendors and the excited chatter of customers. A lone scout, her cloak dusted with desert sand, weaved through the crowd, her hand instinctively resting on the hilt of her dagger."
    ex_rewrite_1 = "A scent of opportunity fills the air. Trust your instincts and a hidden path will reveal itself."
    ex_prompt_1 = "Craft this as a fortune cookie message"

    #second example
    ex_text_2 = "The old lighthouse keeper shuffled down the rickety stairs, his pockets jangling with forgotten keys. He peered out at the churning sea, the salty spray stinging his weathered face. A lone seagull circled overhead, its mournful cry echoing in the wind."
    ex_rewrite_2 = "The lighthouse walls creaked with each of the keeper's slow steps, the rhythm a familiar lullaby to the young apprentice perched by the window. He watched the old man descend, a wizened silhouette framed by the churning sea. The salty spray, invisible from his vantage point, seemed to dance on the keeper's face, etching new lines into the weathered map. The cry of the lone seagull was a melody only the old man understood, a language that spoke of years spent guarding this solitary beacon."
    ex_prompt_2 = "Reimagine this scene from a different perspective, offering a fresh insight into the events"

    #second example
    ex_text_3 = "The population of the earth is estimated to be around 8 billion people, but there are far more insects! Scientists believe that there are over 10 quintillion (that's 10, followed by 18 zeros) individual insects alive at any given time, outweighing all humans and animals combined by a staggering amount."
    ex_rewrite_3 = "Wow, have you ever stopped to think about just how many insects there are? Our planet may be teeming with 8 billion people, but that's nothing compared to the incredible number of insects buzzing, crawling, and flitting around us. Scientists estimate there are over 10 quintillion, that's a 10 with 18 zeros, individual insects alive at any given moment! That's mind-boggling!  And get this, all those insects together weigh more than all the humans and animals on Earth combined.  Pretty incredible, right?"
    ex_prompt_3 = "Rewrite the following as a speech"

    messages = [
        #first example
        {"role": "user", "content": f"{sys_prompt}\n{orig_prefix} {ex_text_1}\n{rewrite_prefix} {ex_rewrite_1}"},
        {"role": "assistant", "content": f"{response_start} {ex_prompt_1}"},

        #second example
        {"role": "user", "content": f"{orig_prefix} {ex_text_2}\n{rewrite_prefix} {ex_rewrite_2}"},
        {"role": "assistant", "content": f"{response_start} {ex_prompt_2}"},

        #actual prompt
        {"role": "user", "content": f"{orig_prefix}{ex_text_3}\n{rewrite_prefix} {ex_rewrite_3}"},
        {"role": "assistant", "content": f"{response_start} {ex_prompt_3}"},
        
        #actual prompt
        {"role": "user", "content": f"{orig_prefix} {row['original_text']}\n{rewrite_prefix} {row['rewritten_text']}"},
        {"role": "assistant", "content": response_start},
    ]
        
    #give it to Mistral
    row['input'] = tokenizer.apply_chat_template(messages, tokenize=False)
    
    return row


    
    
# ds = (
#     ray.data.read_csv('data/3rd_party_ds/llm_dataset_1.csv')
#     .map(get_chat_input)
# )

In [7]:
import pandas as pd

df = pd.read_csv('/home/lawrence/Projects/contests/llm_prompt_recovery/data/3rd_party_ds/gemma-rewrite-nbroad/nbroad-v2.csv')
df.rename(columns={'rewrite_prompt': 'gt_rewrite_prompt'}, inplace=True)
ds = (
    ray.data.from_pandas(df)
    .map(get_chat_input)
).repartition(2) # need to force into multiple blocks <https://discuss.ray.io/t/single-node-4x-gpu-map-batches-only-using-1/12313/2>


Usage stats collection is enabled by default for nightly wheels. To disable this, run the following command: `ray disable-usage-stats` before starting Ray. See https://docs.ray.io/en/master/cluster/usage-stats.html for more details.


2024-04-06 22:03:01,258	INFO worker.py:1752 -- Started a local Ray instance.


In [8]:
df

,id,original_text,gt_rewrite_prompt,rewritten_text
0,lZGdiueMer,"`` Well, there are healthier ways to tell me y...",Rewrite the story where the writer asks the re...,"Well, there are healthier ways to tell me you ..."
1,DfTJVFKrUk,Rory ran his shaky fingers through his wife's ...,Rewrite the essay as a dramatic play,## The Final Curtain\n\n[FADE IN]\n\n**Setting...
2,LmJvKranXK,As I made my way on foot across town to the Po...,Rewrite the story with all the themes and sett...,As I made my way through the Tatooine desert o...
3,PpnqXQAdGH,`` Hello. We come in peace.'' \n \n The first ...,Rewrite the essay if the advanced aliens didn'...,`` Hello. We come in peace.''\n\nThe first enc...
4,qOeXTfqgAM,"`` Karen, what the helllllll izzz...'' says my...",Rewrite the story as a court room drama starri...,The courtroom erupted in an uproar as District...
...,...,...,...,...
2395,ljSaWTXjFc,"`` Dave, have you seen this?'' There is no ans...",Rewrite this as a Shakespearean tragedy,"Alas, poor Dave, a victim of fate's cruel hand..."
2396,nwZeXhsofO,"I posted something like this a while ago, I gu...",Rewrite the story as if you are a famous celeb...,"I'm the epitome of stardom, yet I can't shake ..."
2397,ozaryEVFvK,"`` Oh Darling, it was absolutely dreadful!'' M...",Rewrite the story as an upbeat mystery story (...,"Hey folks, buckle up for a mystery that will h..."
2398,WTvBtRNlQS,Ubwuwuaua! I exclaim shaking my head back and ...,Rewrite the story from the perspective of the ...,Ubwuwuaua. The whispers of the wind dance thro...


In [9]:
# Create a sampling params object.
sampling_params = SamplingParams(max_tokens=100)


# Create a class to do batch inference.
class LLMPredictor:

    def __init__(self):
        self.llm = LLM(
            model=model_name,
            gpu_memory_utilization=0.92,
            max_model_len=1500,
            trust_remote_code=True, 
        )

    def __call__(self, batch: Dict[str, np.ndarray]) -> Dict[str, list]:
        outputs = self.llm.generate(batch["input"], sampling_params)
        
        generated_text = []
        for output in outputs:
            generated_text.append(' '.join([o.text for o in output.outputs]))
        batch["rewrite_prompt"] = generated_text

        return batch

ds = ds.map_batches(
    LLMPredictor,
    # Set the concurrency to the number of LLM instances.
    concurrency=2,
    # Specify the number of GPUs required per LLM instance.
    # NOTE: Do NOT set `num_gpus` when using vLLM with tensor-parallelism
    # (i.e., `tensor_parallel_size`).
    num_gpus=1,
    # Specify the batch size for inference.
    batch_size=2,
)

In [10]:
ds.write_parquet("./data/exp_test/nboard_v2/")

2024-04-06 22:03:04,031	INFO streaming_executor.py:115 -- Starting execution of Dataset. Full log is in /tmp/ray/session_2024-04-06_22-02-59_228988_439769/logs/ray-data.log
2024-04-06 22:03:04,032	INFO streaming_executor.py:116 -- Execution plan of Dataset: InputDataBuffer[Input] -> TaskPoolMapOperator[Map(get_chat_input)] -> AllToAllOperator[Repartition] -> ActorPoolMapOperator[MapBatches(LLMPredictor)] -> TaskPoolMapOperator[Write]



(_MapWorker pid=441463) INFO 04-06 22:03:07 llm_engine.py:87] Initializing an LLM engine with config: model='/home/lawrence/.cache/huggingface/hub/models--mistralai--Mistral-7B-Instruct-v0.2/snapshots/41b61a33a2483885c981aa79e0df6b32407ed873', tokenizer='/home/lawrence/.cache/huggingface/hub/models--mistralai--Mistral-7B-Instruct-v0.2/snapshots/41b61a33a2483885c981aa79e0df6b32407ed873', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=1500, download_dir=None, load_format=auto, tensor_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, device_config=cuda, seed=0)
(_MapWorker pid=441463) INFO 04-06 22:03:15 llm_engine.py:357] # GPU blocks: 3727, # CPU blocks: 2048
(_MapWorker pid=441462) INFO 04-06 22:03:07 llm_engine.py:87] Initializing an LLM engine with config: model='/home/lawrence/.cache/huggingface/hub/models--mistralai--Mistral-7B-Instruct-v0.2/snapshots

- Map(get_chat_input) 1:   0%|          | 0/1 [00:00<?, ?it/s]

- Repartition 2:   0%|          | 0/2 [00:00<?, ?it/s]

Split Repartition 3:   0%|          | 0/2 [00:00<?, ?it/s]

...

Processed prompts: 100%|██████████| 2/2 [00:02<00:00,  1.20s/it] [repeated 6x across cluster]


(MapWorker(MapBatches(LLMPredictor)) pid=441463) WARNING 04-06 22:12:32 scheduler.py:195] Input prompt (1820 tokens) is too long and exceeds limit of 1500 [repeated 24x across cluster]
(MapWorker(MapBatches(LLMPredictor)) pid=441462) WARNING 04-06 22:12:37 scheduler.py:195] Input prompt (1547 tokens) is too long and exceeds limit of 1500 [repeated 10x across cluster]


Processed prompts: 100%|██████████| 2/2 [00:01<00:00,  1.50it/s] [repeated 7x across cluster]
(MapWorker(MapBatches(LLMPredictor)) pid=441463) 


(MapWorker(MapBatches(LLMPredictor)) pid=441462) WARNING 04-06 22:12:43 scheduler.py:195] Input prompt (2670 tokens) is too long and exceeds limit of 1500 [repeated 17x across cluster]


Processed prompts: 100%|██████████| 2/2 [00:02<00:00,  1.21s/it] [repeated 5x across cluster]


In [11]:
train_df = pl.read_parquet("./data/exp_test/nboard_v2/*.parquet").to_pandas()

In [12]:
train_df

,id,original_text,gt_rewrite_prompt,rewritten_text,input,rewrite_prompt
0,KisiIFfldx,"Listen, people seem to think that monsters are...",Rewrite the story as a lighthearted and funny ...,Have you ever been scared by a monster? Oh boy...,"<s>[INST] You are an expert in ""Reverse Prompt...",
1,UyqBvEQOkQ,Like stealing candy from a baby. Mika was the ...,Rewrite the story to have a plot twist halfway...,Like stealing candy from a baby. Mika was the ...,"<s>[INST] You are an expert in ""Reverse Prompt...",Rewrite the text as if it's a confession or a...
2,CwZGipQWvD,`` BILLY MAYS HERE!'' The fireplace burst into...,Rewrite the essay as a philosophical argument ...,The sales process inherently raises ethical qu...,"<s>[INST] You are an expert in ""Reverse Prompt...",
3,pKXubhhwgC,"** [ This is my first writing here, and I am n...",Rewrite the story as a superhero comic from Ma...,**The Crimson Guardian**\n\nIn the heart of a ...,"<s>[INST] You are an expert in ""Reverse Prompt...",
4,IwboJVgxTC,"`` I do n't like it, this things to hot to kee...",Rewrite the scene so that the hijackers are pr...,"The bank was abuzz with life. People lined up,...","<s>[INST] You are an expert in ""Reverse Prompt...",
...,...,...,...,...,...,...
2395,hQtnmdqLsZ,"“ I ’ m telling you, the house has been empty ...",Rewrite the story from the perspective of the ...,The scent of bleach and damp earth lingered in...,"<s>[INST] You are an expert in ""Reverse Prompt...",
2396,xZSLwkEnEc,`` OW! WHAT THE FUCK IS WRONG WITH YOU?'' Jeni...,Rewrite it as a story about a monster who is f...,"In the realm of terror and devastation, a mons...","<s>[INST] You are an expert in ""Reverse Prompt...",Transform the following text into a mythologi...
2397,qlVmrrkVcL,The woman spoke first. `` Help me! He's with t...,Rewrite the essay as a murder mystery where yo...,A murder mystery unfolds as you read this narr...,"<s>[INST] You are an expert in ""Reverse Prompt...",
2398,EbBxajyzHq,Chijin hastened his team again. The barricade ...,Rewrite the essay in the style of an episode o...,"""The Bullpen Bar was buzzing with the scent of...","<s>[INST] You are an expert in ""Reverse Prompt...",


## score the dataset

In [13]:
import numpy as np
from sentence_transformers import SentenceTransformer, util
from sklearn.metrics.pairwise import cosine_similarity

st_model = SentenceTransformer('sentence-transformers/sentence-t5-base')

def get_sharpened_cosine_similarity(text1, text2):
    embeddings1 = st_model.encode(text1)
    embeddings2 = st_model.encode(text2)
    cosine_score = util.cos_sim(embeddings1, embeddings2)
    # print(cosine_score) 
    return (cosine_score[0] ** 3).numpy()[0]

def calc_prompt_similarity(row):
    return get_sharpened_cosine_similarity(row['gt_rewrite_prompt'], row['rewrite_prompt'])

train_df['score'] = train_df.apply(lambda row: calc_prompt_similarity(row), axis=1)

train_df['score'].mean()

0.48421794

In [14]:
train_df.to_csv('./data/predictions/nboard_v2.csv')